# विषय सूची
- [मार्गदर्शनाबद्दल](../../../../code/01.Introduce)
- [सेटअप](../../../../code/01.Introduce)
- [अमर्यादित निर्मिती](../../../../code/01.Introduce)
- [फाय 3 साठी बोलणे](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [निवड](../../../../code/01.Introduce)
- [विचारांची साखळी](../../../../code/01.Introduce)
- [JSON निर्मिती](../../../../code/01.Introduce)
- [HTML निर्मिती](../../../../code/01.Introduce)


# Guidance बद्दल  
Guidance ही कोणत्याही भाषा मॉडेल (LM) च्या आउटपुट्स नियंत्रित करण्यासाठी वापरली जाणारी एक सिद्ध ओपन-सोर्स Python लायब्ररी आहे. एका API कॉलसह, तुम्ही (Python मध्ये) मॉडेलने अनुसरण कराव्या लागणाऱ्या अचूक प्रोग्रामॅटिक अटी व्यक्त करू शकता आणि JSON, Python, HTML, SQL किंवा वापराच्या प्रकरणासाठी आवश्यक असलेल्या कोणत्याही संरचनेत संरचित आउटपुट तयार करू शकता.

Guidance पारंपरिक prompting तंत्रांपेक्षा वेगळी आहे. ती मॉडेलला टोकन बाय टोकन inference स्तरावर मार्गदर्शन करून अटी लागू करते, उच्च दर्जाचे आउटपुट तयार करते आणि अत्यंत संरचित परिस्थितींमध्ये वापरताना खर्च आणि विलंब 30–50% पर्यंत कमी करते.

Guidance बद्दल अधिक जाणून घेण्यासाठी, [GitHub वरील सार्वजनिक रिपॉझिटरी](https://github.com/guidance-ai/guidance) ला भेट द्या किंवा Microsoft Build मधील [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) पहा.


# सेटअप
1. Guidance `pip install guidance --pre` वापरून इंस्टॉल करा
2. Azure मध्ये Phi 3.5 mini endpoint डिप्लॉय करा. यासाठी https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml या लिंकवर जा आणि "Deploy" बटणावर क्लिक करा
3. तुमच्या endpoint चा API key `AZURE_PHI3_KEY` नावाच्या environment variable मध्ये आणि URL `AZURE_PHI3_URL` नावाच्या environment variable मध्ये स्टोअर करा


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# अमर्यादित निर्मिती
`gen()` फंक्शन वापरून मजकूर कोणत्याही मर्यादेशिवाय तयार केला जाऊ शकतो. हे Guidance शिवाय मॉडेल वापरण्यासारखेच आहे.

## चॅट स्वरूपन
अनेक चॅट मॉडेल्सप्रमाणे, Phi-3 वापरकर्ता आणि सहाय्यक यांच्यातील संदेश एका विशिष्ट स्वरूपात अपेक्षित करते. Guidance Phi-3 च्या चॅट टेम्पलेटला समर्थन देते आणि तुमच्यासाठी चॅट स्वरूपन व्यवस्थापित करेल. चॅट टर्न तयार करण्यासाठी, संभाषणातील प्रत्येक भाग `with user()` किंवा `with assistant()` ब्लॉकमध्ये ठेवा. `with system()` ब्लॉकचा वापर सिस्टम संदेश सेट करण्यासाठी केला जाऊ शकतो.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## टोकन बचत
अत्यंत संरचित परिस्थितींमध्ये, Guidance टोकन वगळू शकते आणि फक्त आवश्यक टोकन तयार करू शकते, ज्यामुळे कार्यक्षमता सुधारते, कार्यक्षमता वाढते आणि API खर्चाची बचत होते. या नोटबुकमध्ये तयार केलेले टोकन हायलाइट केलेल्या पार्श्वभूमीसह दाखवले जातात. जबरदस्तीने तयार केलेले टोकन हायलाइटिंगशिवाय दाखवले जातात आणि इनपुट टोकनइतकेच खर्च करतात, जे आउटपुट टोकनच्या खर्चाच्या एक तृतीयांश म्हणून अंदाजित केले जातात.

*टीप:* अनियंत्रित जनरेशनसह पहिल्या उदाहरणात कोणतेही टोकन जबरदस्तीने तयार करता आले नाहीत कारण आम्ही कोणतीही बंधने दिली नव्हती.


# Phi 3 साठी बोलणे  
मार्गदर्शनासह, तुम्ही सहजपणे मॉडेलच्या प्रतिसादांमध्ये मजकूर समाविष्ट करू शकता. जर तुम्हाला मॉडेलच्या आउटपुटला विशिष्ट दिशेने मार्गदर्शन करायचे असेल तर हे उपयुक्त ठरू शकते.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Regex वापरून मर्यादित करणे
मागील उदाहरणात, Phi 3 ने `Canberra` या उत्तरानंतर पुढील स्पष्टीकरणे दिली. मॉडेलचे आउटपुट नेमके एक शब्दावर मर्यादित करण्यासाठी regex वापरता येतो.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# अनेक पर्यायांमधून निवड करणे
जेव्हा काही संभाव्य पर्याय माहित असतात, तेव्हा तुम्ही `select()` फंक्शन वापरून मॉडेलला पर्यायांच्या यादीतून निवड करण्यास सांगू शकता.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` वापरल्यावर फक्त `Can` टोकन तयार झाले. कारण `Canberra` हा एकमेव पर्याय आहे जो उत्तर पूर्ण करू शकतो, उर्वरित टोकन जबरदस्तीने तयार करण्यात आले.


# विचारांची साखळी
विचारांची साखळी ही एक तंत्र आहे जी मॉडेलला समस्या टप्प्याटप्प्याने प्रक्रिया करण्यास प्रोत्साहन देऊन त्याच्या आउटपुटची गुणवत्ता सुधारण्यास मदत करू शकते. सामान्यतः, अंतिम उत्तर मिळवण्यासाठी अनेक प्रॉम्प्ट टर्न्स आवश्यक असतात. प्रथम, मॉडेलला टप्प्याटप्प्याने विचार करण्याचे निर्देश द्या. त्यानंतर, मॉडेलला अंतिम उत्तर देण्यासाठी पुन्हा प्रॉम्प्ट करा. स्टँडर्ड चॅट इनफरन्स API वापरताना, यासाठी 2 API कॉल्स लागतात, आणि मॉडेलने तयार केलेल्या "विचारांची साखळी" दोनदा चार्ज होते – एकदा आउटपुट टोकन्स म्हणून जेव्हा मॉडेलने ती तयार केली, आणि नंतर इनपुट टोकन्स म्हणून दुसऱ्या कॉलसाठी. Guidance वापरून, संपूर्ण बहु-टप्पा प्रक्रिया एका API कॉलचा भाग म्हणून प्रक्रिया केली जाते आणि चार्ज केली जाते, ज्यामुळे खर्च आणि विलंब कमी होतो.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON निर्मिती
Guidance चा उपयोग JSON schema किंवा pydantic मॉडेलशी सुसंगत JSON तयार करण्याची खात्री देण्यासाठी केला जाऊ शकतो, जसे की येथे दाखवलेला वापरकर्ता प्रोफाइल schema.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML निर्मिती

Guidance चा उपयोग कोड तयार करण्यासाठी आणि प्रोग्रामिंग भाषेतील व्याकरणात्मक आवश्यकता पूर्ण करण्यासाठी केला जाऊ शकतो. या विभागात, आपण अतिशय साध्या HTML वेबपेजेस लिहिण्यासाठी एक छोटा Guidance प्रोग्राम तयार करू.

आपण वेबपेजला लहान विभागांमध्ये विभागू, प्रत्येकाचा स्वतःचा Guidance फंक्शन असेल. नंतर हे विभाग आमच्या अंतिम फंक्शनमध्ये एकत्र करून HTML वेबपेज तयार करू.
यानंतर आपण Guidance-सक्षम मॉडेलवर Azure AI मध्ये ही फंक्शन चालवू.

*टीप:* हे पूर्ण-वैशिष्ट्यीकृत HTML जनरेटर होणार नाही; उद्दिष्ट हे आहे की आपण आपल्या वैयक्तिक गरजांसाठी संरचित आउटपुट कसे तयार करू शकता हे दाखवणे.

आपण Guidance मधून आवश्यक गोष्टी आयात करून सुरुवात करू:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML वेबपृष्ठे अत्यंत संरचित असतात, आणि आम्ही Guidance चा वापर करून त्या पृष्ठाच्या भागांना 'जबरदस्तीने' लागू करू.  
जेव्हा आम्ही मॉडेलकडून विशिष्ट मजकूर आवश्यक करतो, तेव्हा आम्हाला खात्री करावी लागते की त्यामध्ये कोणतेही टॅग असू शकत नाहीत - म्हणजेच, आम्हाला '<' आणि '>' या अक्षरे वगळावी लागतील.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

आम्ही नंतर ही फंक्शन वापरून मनमानी HTML टॅगमध्ये मजकूर तयार करू शकतो:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

आता आपण पृष्ठ शीर्षक तयार करूया. याचा एक भाग म्हणून, आपल्याला पृष्ठाचे शीर्षक तयार करणे आवश्यक आहे:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML पृष्ठाच्या मुख्य भागात शीर्षके आणि परिच्छेद असतील.  
आपण प्रत्येकासाठी एक फंक्शन परिभाषित करू शकतो:  


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

आता HTML च्या मुख्य भागाची व्याख्या करण्यासाठी फंक्शन तयार करा.  
हे `select()` चा वापर `recurse=True` सह करते, ज्यामुळे अनेक शीर्षके आणि परिच्छेद तयार होतात:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

आता आपण त्या फंक्शनकडे येतो जे संपूर्ण HTML पृष्ठ तयार करते.  
आम्ही HTML सुरुवातीचा टॅग जोडतो, त्यानंतर हेडर तयार करतो, मग बॉडी तयार करतो, आणि शेवटी HTML समाप्तीचा टॅग जोडतो:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

आम्ही एक वापरकर्ता-अनुकूल आवरण प्रदान करतो, ज्यामुळे आम्हाला खालील गोष्टी करता येतील:
- जनरेशनचे तापमान सेट करणे
- Model ऑब्जेक्टमधून तयार केलेले पृष्ठ कॅप्चर करणे


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

आम्ही नंतर आउटपुट एका फाइलमध्ये लिहू शकतो:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

आणि [परिणाम पहा](../../../../code/01.Introduce/sample_page.html).



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
